In [14]:
#读取必备的数据
def read_data():
    train_dataset = load_dataset('csv',data_files = './train.csv',split = 'train')
    #print(train_dataset)
    #print(len(train_dataset))
    #读取测试集
    test_dataset = load_dataset('csv',data_files = './test.csv',split = 'train')
    #print(test_dataset)
    #print(len(test_dataset))
    #读取验证集
    validation_dataset = load_dataset('csv',data_files = './validation.csv',split = 'train')
    #print(validation_dataset)
    #print(len(validation_dataset))
    return train_dataset,test_dataset ,validation_dataset
#read_data()

In [52]:
#中文分类
# 导入工具包
import os
import sys
import torch.nn as nn
# 添加环境变量
os.environ['TRANSFORMERS_OFFLINE'] = '0'
import torch
from datasets import load_dataset
from transformers import BertTokenizer, BertModel
from torch.optim import AdamW
import time
from torch.utils.data import DataLoader
from tqdm import tqdm
#加载分词器
my_tokenizer = BertTokenizer.from_pretrained(r'D:\bert')

# 2 加载模型
my_model = BertModel.from_pretrained(r'D:\bert').to(device)#和自动模型区别只在于导包不相同
#print(my_model)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

Some weights of the model checkpoint at D:\bert were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda


In [24]:
#获取dataloader
#用于dataset进行处理
def collate_fn(data):
    #获取一个批次样本中的句子和标签
    sentence = [value['text'] for value in data]
    label = [value['label'] for value in data]
    print()
    #print(sentence)
    #print(label)
    #对一个批次的句子进行张量表示
    data = my_tokenizer.batch_encode_plus(batch_text_or_text_pairs=sentence,
                                          padding = 'max_length',
                                          truncation = True,
                                          max_length = 200,
                                          return_tensors = 'pt',
                                          )
    input_ids = data['input_ids']
    token_type_ids = data['token_type_ids']
    attention_mask = data['attention_mask']
    labels_y = torch.tensor(label,dtype = torch.long)
    return input_ids,token_type_ids,attention_mask,labels_y
    
def get_dataloader():
    train_dataset,_,_ = read_data()
    #对上述进行dataloader封装
    train_dataloader = DataLoader(dataset =train_dataset,batch_size = 8,collate_fn = collate_fn,drop_last = True,shuffle=True)
    #封装中取元素 需要进行迭代
    #for input_ids,token_type_ids,attention_mask,labels_y in train_dataloader:
       # print("test")
        #break
    return train_dataloader
#get_dataloader()

In [27]:
#=自定义下游任务网络模型 自定义的微调网络参数总数大于0.5倍的训练数据量小于10倍的
class Mymodel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(768,2)
    def forward(self,input_ids,token_type_ids,attention_mask):
        #预训练模型的参数不更新
        with torch.no_grad():
            output = my_model(input_ids = input_ids,
                           attention_mask = attention_mask,
                           token_type_ids = token_type_ids)
        #将bert编码后的结果送入输出层
        result = self.fc(output['pooler_output'])
        return result
train_dataloader = get_dataloader()
mymodel = Mymodel()
for input_ids,token_type_ids,attention_mask,labels_y in train_dataloader:
    print("test")
    output = mymodel(input_ids,token_type_ids,attention_mask)
    print(output.shape)
    break


test
torch.Size([8, 2])


In [58]:
#模型训练
def train_model():
    #实例化数据源对象
    train_dataset = load_dataset('csv',data_files = './train.csv',split = 'train')
    #将上述的数据进行封装
    train_dataloader = DataLoader(dataset =train_dataset,batch_size = 8,collate_fn = collate_fn,drop_last = True,shuffle=True)
    #实例化模型
    mymodel = Mymodel().to(device)
    #实例化优化器
    my_optimizer = AdamW(mymodel.parameters(),lr = 5e-4)
    #实例化损失函数
    my_criterion = torch.nn.CrossEntropyLoss()
    # 不训练预训练模型 只让预训练模型计算数据特征 不需要计算梯度
    for param in my_model.parameters():
        param.requires_grad_(False)#删除的时候最原始的模型会进行更新
    #设置训练参数
    epochs = 1
    #设置训练模式
    mymodel.train()
    #开始训练
    for epoch in range(epochs):
        start_time = time.time()
        #开启内部数据迭代
        for idx,(input_ids,token_type_ids,attention_mask,labels_y) in enumerate(tqdm(train_dataloader),start = 1):
            #数据送入模型
            input_ids = input_ids.to(device)
            token_type_ids = token_type_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels_y = labels_y.to(device)
            output = mymodel(input_ids,token_type_ids,attention_mask)
            #print(output)
            #计算损失
            myloss = my_criterion(output,labels_y)
            #print(myloss)
            #梯度清0
            my_optimizer.zero_grad()
            #反向传播
            myloss.backward()
            #梯度更新
            my_optimizer.step()
            if idx % 50 == 0:
                out = output.argmax(dim=1) # [8,2] --> (8,)
                accuracy = (out == labels_y).sum().item() / len(labels_y)
                print(f'轮次:{epoch+1} 迭代数:{idx} 损失:{myloss.item():.6f} 准确率:{accuracy:.3f} 时间:{time.time()-start_time}s')

        # 每个轮次保存模型
        torch.save(mymodel.state_dict(), './my_model_%d.bin' % (epoch + 1))

train_model()

  0%|          | 2/1200 [00:00<01:22, 14.48it/s]

  0%|          | 4/1200 [00:00<01:56, 10.24it/s]

  0%|          | 6/1200 [00:00<02:07,  9.35it/s]

  1%|          | 8/1200 [00:00<02:13,  8.90it/s]

  1%|          | 10/1200 [00:01<02:18,  8.59it/s]

...

100%|█████████▉| 1196/1200 [02:24<00:00,  7.98it/s]

100%|█████████▉| 1198/1200 [02:24<00:00,  7.88it/s]

100%|█████████▉| 1199/1200 [02:24<00:00,  8.02it/s]

100%|██████████| 1200/1200 [02:25<00:00,  8.28it/s]

轮次:1 迭代数:1200 损失:0.314288 准确率:0.875 时间:144.99995040893555s


In [63]:
def test_model():
    #实例化数据源对象
    test_dataset = load_dataset('csv',data_files = './test.csv',split = 'train')
    #将上述的数据进行封装
    test_dataloader = DataLoader(dataset =test_dataset,batch_size = 8,collate_fn = collate_fn,drop_last = True,shuffle=True)
    #加载模型
    my_model = Mymodel().to(device)
    my_model.load_state_dict(torch.load('./my_model_1.bin'))
    #定义测试超参数
    total = 0
    acc_num = 0
    my_model.eval()
    #开始测试
    for idx,(input_ids,token_type_ids,attention_mask,labels_y) in enumerate(tqdm(test_dataloader),start = 1):
        input_ids = input_ids.to(device)
        token_type_ids = token_type_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels_y = labels_y.to(device)
        with torch.no_grad():
            output = my_model(input_ids,token_type_ids,attention_mask)
        #计算正确的样本个数
        out = output.argmax(dim=1) # [8,2] --> (8,)
        acc_num = acc_num + (out == labels_y).sum().item() 
        total = total+len(labels_y)
        if idx %5 ==0:
            print(acc_num/total,end =' ')
            print(my_tokenizer.decode(input_ids[0],skip_special_tokens=True),end=' ')
            print('预测值 真实值:',out[0].item(),labels_y[0].item())
test_model()

  1%|          | 1/150 [00:00<00:18,  8.00it/s]

  2%|▏         | 3/150 [00:00<00:18,  8.00it/s]

  3%|▎         | 5/150 [00:00<00:18,  7.98it/s]


0.85 整 个 酒 店 一 股 羊 肉 的 臊 味, 11 月 了 还 没 暖 气, 有 空 调, 居 然 没 有 暖 风, 挨 冻....... 找 遍 整 个 房 间 没 找 到 酒 店 任 何 服 务 电 话, 只 有 一 个 按 摩 的...... 半 夜 被 汽 车 喇 叭 惊 醒, 以 为 窗 户 没 关 好, 后 来 发 现 关 的 很 严...... 只 有 开 着 电 视, 昏 昏 沉 沉 的 居 然 听 到 了 火 车 声...... 预测值 真实值: 0 0



  5%|▍         | 7/150 [00:00<00:17,  8.00it/s]

...


0.8603448275862069 外 观 很 有 特 点 ， 长 城 外 观 ， 显 然 白 天 为 了 省 电 ， 大 堂 不 开 灯 。 。 。 似 乎 感 觉 上 入 住 的 人 数 有 点 少 ， 另 外 就 是 入 住 手 续 有 点 意 思 ， 开 什 么 入 住 通 知 书 等 等 。 。 。 预测值 真实值: 0 1



 98%|█████████▊| 147/150 [00:18<00:00,  8.11it/s]

 99%|█████████▉| 149/150 [00:18<00:00,  8.05it/s]

100%|██████████| 150/150 [00:18<00:00,  8.06it/s]

0.8583333333333333 我 是 看 了 电 视 剧 觉 得 还 不 错 ， 所 以 又 买 了 书 看 ， 感 觉 书 还 是 更 好 一 些 。 毕 竟 有 些 心 理 描 写 不 是 电 视 剧 能 比 拟 的 。 可 能 这 本 书 的 主 人 公 和 我 的 年 龄 差 不 多 ， 所 以 在 其 中 有 时 感 觉 和 生 活 太 贴 近 了 ， 所 以 和 主 人 公 的 情 绪 一 起 不 停 的 起 起 落 落 。 对 于 我 们 这 个 讲 求 独 立 的 新 的 一 代 人 的 婚 姻 来 讲 ， 比 父 辈 们 面 临 了 更 多 的 选 择 和 困 扰 。 主 人 公 的 经 历 对 我 们 现 实 中 的 小 夫 妻 来 说 也 是 一 个 启 示 ， 希 望 从 中 能 找 出 自 己 相 似 的 缺 点 和 不 足 。 预测值 真实值: 1 1
